In [3]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from dateutil.parser import parse
import time
import os
import pickle

<h1># 03 - Wetterdaten: Vorbereitung - Schritt 1 </h1>
<hr>
<p><b>Hinweis:</b> Dateifpade sind absolut angegeben und müssen entsprechend der eigenen Verzeichnisstruktur angepasst werden!</p>
<hr>
<p>Hier werden die Wetterdaten nur auf die benötigte Zeiträume gefiltert</p>
<hr>

In [4]:
# Anzahl der angezeigten Zeilen in JNP einstellen
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
# Pickl-Dateipfade vorbereiten:

data_path_wma = '/home/paul/python_projects/masterthesis/data/wmm_data/'
pkl_file_wma = 'daten_wmm_all_prepared.p'

path_to_downloaded_weather_files = '/home/paul/python_projects/masterthesis/data/wetter/downloaded/'
pkl_file_wetter_raw_dwn = 'daten_wetter_all_raw.p'

path_to_working_weather_files = '/home/paul/python_projects/masterthesis/data/wetter/'

In [6]:
# Pickl-Dateien einlesen:

df_wma_prepared = pickle.load(open(data_path_wma + pkl_file_wma, 'rb')) # Ergebnisdaten
df_wetter_raw = pickle.load(open(path_to_downloaded_weather_files + pkl_file_wetter_raw_dwn, 'rb')) # Wetterdaten Roh

In [7]:
df_wma_prepared.head()

,Jahr,Ort,Geschlecht,Vorname,Nachname,Platz,Datum,Startzeit,Datum_Startzeit_UTC,T_KM_5,S_KM_5,T_KM_10,S_KM_10,T_KM_15,S_KM_15,T_KM_20,S_KM_20,T_KM_HM,S_KM_HM,T_KM_25,S_KM_25,T_KM_30,S_KM_30,T_KM_35,S_KM_35,T_KM_40,S_KM_40,T_KM_FN,S_KM_FN,ZZ_INVALID
0,2007,Berlin,M,Haile,Gebrselassie,1,2007-09-30,09:15:00,2007-09-30 07:15:00,00:14:44,884.0,00:29:27,1767.0,00:44:16,2656.0,00:59:10,3550.0,01:02:29,3749.0,01:14:05,4445.0,01:28:56,5336.0,01:43:38,6218.0,01:58:08,7088.0,02:04:26,7466.0,F
1,2007,Berlin,M,Abel,Kirui,2,2007-09-30,09:15:00,2007-09-30 07:15:00,00:14:57,897.0,00:30:00,1800.0,00:45:06,2706.0,01:00:16,3616.0,01:03:41,3821.0,01:15:22,4522.0,01:30:01,5401.0,01:44:52,6292.0,02:00:09,7209.0,02:06:51,7611.0,F
2,2007,Berlin,M,Salim,Kipsang,3,2007-09-30,09:15:00,2007-09-30 07:15:00,00:14:57,897.0,00:30:01,1801.0,00:45:06,2706.0,01:00:16,3616.0,01:03:41,3821.0,01:15:22,4522.0,01:30:05,5405.0,01:44:53,6293.0,02:00:27,7227.0,02:07:29,7649.0,F
3,2007,Berlin,M,Philip,Manyim,4,2007-09-30,09:15:00,2007-09-30 07:15:00,00:14:58,898.0,00:30:00,1800.0,00:45:08,2708.0,01:00:16,3616.0,01:03:41,3821.0,01:15:22,4522.0,01:30:01,5401.0,01:44:53,6293.0,02:00:26,7226.0,02:08:01,7681.0,F
4,2007,Berlin,M,Mesfin,Adimasu,5,2007-09-30,09:15:00,2007-09-30 07:15:00,00:14:57,897.0,00:30:00,1800.0,00:45:06,2706.0,01:00:16,3616.0,01:03:41,3821.0,01:15:22,4522.0,01:30:05,5405.0,01:45:14,6314.0,02:01:58,7318.0,02:09:49,7789.0,F


In [8]:
df_wetter_raw.head()

,ORT,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP
0,Berlin,10382099999,2011-01-01T00:20:00,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1"
1,Berlin,10382099999,2011-01-01T00:50:00,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1"
2,Berlin,10382099999,2011-01-01T01:20:00,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1"
3,Berlin,10382099999,2011-01-01T01:50:00,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1"
4,Berlin,10382099999,2011-01-01T02:20:00,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1"


In [9]:
# Kopie der Wetterdaten für die weitere Verarbeitung erstellen
df_wetter_1 = df_wetter_raw.copy()

# Kopie der Marathondaten mit notwendigen Spalten erstellen
df_wma_1 = df_wma_prepared[['Jahr','Ort','Datum','Datum_Startzeit_UTC']].copy()
df_wma_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00


In [10]:
# Neue Spalte DATE_TIME aus DATE erzeugen.
df_wetter_1['DATE_TIME'] = df_wetter_1['DATE'].astype('datetime64')
# Uhrzeit aus der DATE Spalte entfernen, um später nur die relevanten Daten herauszufiltern
df_wetter_1['DATE'] = df_wetter_1['DATE_TIME'].dt.normalize()
df_wetter_1.head()

,ORT,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME
0,Berlin,10382099999,2011-01-01,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1",2011-01-01 00:20:00
1,Berlin,10382099999,2011-01-01,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1",2011-01-01 00:50:00
2,Berlin,10382099999,2011-01-01,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1",2011-01-01 01:20:00
3,Berlin,10382099999,2011-01-01,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1",2011-01-01 01:50:00
4,Berlin,10382099999,2011-01-01,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0010,1",2011-01-01 02:20:00


In [11]:
# Mithilfe der WMA-Daten nur die relevanten Zeiträume der Wetterdaten filtern (mittels JOIN)
df_wetter_2 = pd.merge(left=df_wma_1, right=df_wetter_1, how='inner', left_on=['Ort','Datum'], right_on=['ORT','DATE'])
df_wetter_2.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,ORT,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,Berlin,10382099999,2007-09-30,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,Berlin,10382099999,2007-09-30,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,Berlin,10382099999,2007-09-30,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,Berlin,10382099999,2007-09-30,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,Berlin,10382099999,2007-09-30,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0120,1",2007-09-30 02:20:00


In [12]:
# Nicht benötigte (doppelte) Spalten entfernen
df_wetter_3 = df_wetter_2.drop(columns=['ORT','DATE']).copy()
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0120,1",2007-09-30 02:20:00


In [13]:
# Spalten hinzufügen die die Zeitspanne des Lauf repräsentieren
df_wetter_3['RANGE_DT_START'] = df_wetter_3['Datum_Startzeit_UTC'] 
df_wetter_3['RANGE_DT_END'] = None
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:20:00,2007-09-30 07:15:00,None
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:50:00,2007-09-30 07:15:00,None
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:20:00,2007-09-30 07:15:00,None
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:50:00,2007-09-30 07:15:00,None
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0120,1",2007-09-30 02:20:00,2007-09-30 07:15:00,None


In [14]:
# Startzeitstpanne runden - 30 min
df_wetter_3['RANGE_DT_START'] = df_wetter_3['RANGE_DT_START'].dt.round('30min')

In [15]:
df_wetter_3.loc[(df_wetter_3.Ort == 'London') & (df_wetter_3.Jahr == '2019')].head(10)

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
42900,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0100,1",2019-04-28 00:20:00,2019-04-28 09:00:00,None
42901,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0100,1",2019-04-28 00:50:00,2019-04-28 09:00:00,None
42902,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 01:20:00,2019-04-28 09:00:00,None
42903,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 01:50:00,2019-04-28 09:00:00,None
42904,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0080,1",2019-04-28 02:20:00,2019-04-28 09:00:00,None
42905,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 02:50:00,2019-04-28 09:00:00,None
42906,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 03:20:00,2019-04-28 09:00:00,None
42907,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 03:50:00,2019-04-28 09:00:00,None
42908,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 04:20:00,2019-04-28 09:00:00,None
42909,2019,London,2019-04-28,2019-04-28 09:10:00,03768399999,4,51.505278,0.055278,5.79,"CITY, UK",FM-15,V020,"+0090,1",2019-04-28 04:50:00,2019-04-28 09:00:00,None


In [16]:
# Startendzeitspanne berechnen: Es werden 3h zu der Startzeit hinzugerechnet, weil die Läufe zwischen 2 und 3 Stunden dauern (im Durchschnitt 2,5h)
df_wetter_3['RANGE_DT_END'] = df_wetter_3['RANGE_DT_START'] + dt.timedelta(hours=3)
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 00:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 01:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0120,1",2007-09-30 02:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00


In [17]:
df_wetter_4 = df_wetter_3.copy()

In [18]:
# Datensätze rausfiltern die nicht in der Range der relevanten Wetterzeiten liegen
# df_wetter_4.count() # 80980
# df_wetter_4.loc[(df_wetter_4['DATE_TIME'] >= df_wetter_4.RANGE_DT_START) & (df_wetter_4['DATE_TIME'] <= df_wetter_4.RANGE_DT_END)].count() # 11360
# df_wetter_4.loc[(df_wetter_4['DATE_TIME'] >= df_wetter_4.RANGE_DT_START) & (df_wetter_4['DATE_TIME'] <= df_wetter_4.RANGE_DT_END) & (df_wetter_4.Ort == 'Berlin') & (df_wetter_4.Jahr == '2007')] 
df_wetter_5 = df_wetter_4.loc[(df_wetter_4['DATE_TIME'] >= df_wetter_4.RANGE_DT_START) & (df_wetter_4['DATE_TIME'] <= df_wetter_4.RANGE_DT_END)].copy()

In [19]:
# Dubplikate entfernen (auf Grund des Joins) und Index neuerstellen
df_wetter_5.drop_duplicates(inplace=True)
df_wetter_5.reset_index(drop=True, inplace=True)
df_wetter_5.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME,RANGE_DT_START,RANGE_DT_END
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 07:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 07:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 08:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 08:50:00,2007-09-30 07:00:00,2007-09-30 10:00:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0150,1",2007-09-30 09:20:00,2007-09-30 07:00:00,2007-09-30 10:00:00


In [20]:
# Range-Spalten entfernen
df_wetter_5.drop(columns=['RANGE_DT_START','RANGE_DT_END'],inplace=True)
df_wetter_5.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,TMP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0130,1",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0140,1",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"+0150,1",2007-09-30 09:20:00


In [21]:
# Dataframe serialisieren (Pickle) 
pickle.dump(df_wetter_5, open(path_to_working_weather_files + 'daten_wetter_step_1.p', "wb"))